In [41]:
from langchain.utilities import GoogleSerperAPIWrapper, BashProcess
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain, LLMMathChain


In [42]:
search = GoogleSerperAPIWrapper()
llm = OpenAI(temperature=0)


In [43]:
llm_math_chain = LLMMathChain(llm=llm, verbose=True)

In [44]:
bash = BashProcess()

In [45]:
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="Bash",
        func=bash.run,
        description="useful for when you need to execute bash commands"
    ),
]


In [46]:
prefix = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:"""
suffix = """Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Args"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

print(prompt.template)



Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

Search: useful for when you need to answer questions about current events
Calculator: useful for when you need to answer questions about math
Bash: useful for when you need to execute bash commands

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator, Bash]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Args"

Question: {input}
{agent_scratchpad}


In [47]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)



In [52]:
agent_executor.run("What is the value of my free memory devide to population of viet nam?")



> Entering new AgentExecutor chain...

Thought: I need to figure out the value of my free memory and the population of Vietnam
Action: Search
Action Input: Population of Vietnam
Observation: 97.47 million (2021)
Thought: I need to figure out the value of my free memory
Action: Bash
Action Input: free -m
Observation:               total        used        free      shared  buff/cache   available
Mem:           3932        2456         191           0        1284        1182
Swap:             0           0           0

Thought: I now know the final answer
Final Answer: Arrr, me free memory be 191 divided by 97.47 million, which be 0.00195!

> Finished chain.


'Arrr, me free memory be 191 divided by 97.47 million, which be 0.00195!'